In [2]:
import random
import operator
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools


pset = gp.PrimitiveSet("main", 9)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
a=4
pset.addTerminal(4)


pset.renameArguments(ARG0="thickness")
pset.renameArguments(ARG1="size")
pset.renameArguments(ARG2="shape")
pset.renameArguments(ARG3="ma")
pset.renameArguments(ARG4="se")
pset.renameArguments(ARG5="bn")
pset.renameArguments(ARG6="bc")
pset.renameArguments(ARG7="nn")
pset.renameArguments(ARG8="mitoses")


# expr = gp.genFull(pset, min_=1, max_=5)

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)



toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=10)



x=toolbox.population()
print(x[1][1])
print(x[9][9])

tree = gp.PrimitiveTree(expr)
print(tree)



max(max(max(ma, 4), max(mitoses, se)), mul(add(shape, shape), max(thickness, ma)))
mul(max(mul(bn, nn), add(nn, shape)), add(mul(mitoses, max(thickness, thickness)), max(shape, 4)))
add(max(ma, size), max(4, bc))


C:\Users\lethalTranquilizer\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [10]:
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

def Make_Batches():
    data = pd.read_csv('Dataset_MLAI.txt',sep = ',',header = None)
    #display(data)
    data[6] =  pd.to_numeric(data[6], errors='coerce') 
    # drop nas
    data = data.dropna()

    x_features = data.iloc[:,1:10]
    y_features = data.iloc[:,10]
    y_list = list(y_features.values.T.ravel())
    x_array = x_features.values

    scaler = preprocessing.StandardScaler()
    scaled_x_l = []
    y_train_l = []
    for i in range(12):
        x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state= random.randint(1,100),shuffle = True)
        scaled_x_l.append(scaler.fit_transform(x_train))
        y_train_l.append(y_train)
    return(scaled_x_l,y_train_l)

scaled_x_lit,y_train_lit = Make_Batches()
print(scaled_x_lit, y_train_lit)


[array([[-0.93541435, -0.56380401, -0.5634891 , -0.69684919,  3.16133749,
        -0.65654043, -0.9009982 , -0.48210508, -0.37907125],
       [ 0.40089186, -0.56380401, -0.5634891 ,  0.98907626, -0.07710579,
        -0.65654043, -0.50349899, -0.48210508, -0.37907125],
       [-0.26726124, -0.56380401, -0.5634891 , -0.69684919, -0.53974055,
        -0.65654043, -0.10599979, -0.48210508, -0.37907125],
       [-0.93541435, -0.56380401, -0.23839923, -0.3596641 , -0.53974055,
        -0.65654043, -0.10599979, -0.48210508, -0.37907125],
       [-0.26726124, -0.26706506, -0.23839923, -0.02247901, -0.53974055,
        -0.65654043, -0.9009982 , -0.48210508, -0.37907125],
       [-0.26726124, -0.56380401, -0.5634891 , -0.69684919, -0.53974055,
        -0.65654043, -0.50349899, -0.48210508, -0.37907125],
       [-0.93541435, -0.56380401, -0.5634891 , -0.69684919, -0.53974055,
        -0.65654043, -0.9009982 , -0.48210508, -0.37907125],
       [ 2.07127462,  0.6231518 ,  0.73687036, -0.02247901,  

In [22]:
def treeBatchFitness(tree, x, y):
    function = gp.compile(str(tree), pset)
    fitnessS=[]
    scaled_x_lit,y_train_lit = x,y
    for j in range(12):
        tn=0
        tp=0
        fn=0
        fp=0
        listRand=[]
        for i in range(30):
            if(function((scaled_x_lit[j])[i,0],(scaled_x_lit[j])[i,1],(scaled_x_lit[j])[i,2],(scaled_x_lit[j])[i,3],(scaled_x_lit[j])[i,4],(scaled_x_lit[j])[i,5],(scaled_x_lit[j])[i,6],(scaled_x_lit[j])[i,7],(scaled_x_lit[j])[i,8])<0):
                listRand.append(2)
            else:
                listRand.append(4)
#         print("Batch "+str(j+2)+" : ")

        for n in range(30):
            if(listRand[n]==(y_train_lit[j])[n]):
                if((y_train_lit[j])[n]==2):
                    tn=tn+1
                else:
                    tp=tp+1
            else:
                if(listRand[n]==2):
                    fn=fn+1
                else:
                    fp=fp+1
#         print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
        prec=tp/(tp+fp)
        recall=tp/(tp+fn)
        fitness=(2*prec*recall)/(prec+recall)
        fitnessS.append(fitness*100)
    return(fitnessS)

In [23]:
def Pearson_evaluation(tree1 ,tree2):
    lsum_product = []; lsumx_product = [];  lsumy_product = [];
    f_tree1 = Tree_fitness(tree1,scaled_x_lit,y_train_lit)
    f_tree2 = Tree_fitness(tree2,scaled_x_lit,y_train_lit)
    x_mean = np.mean(f_tree1)
    y_mean = np.mean(f_tree2)
    for j,k in zip(f_tree1,f_tree2):
        sub_x = j-x_mean; sub_y = k-y_mean
        product = (sub_x)*(sub_y)
        square_x = sub_x**2;square_y = sub_y**2

        lsum_product.append(product); lsumx_product.append(square_x); lsumy_product.append(square_y);
    x = np.sum(lsum_product)
    y = np.sqrt(np.sum(lsumx_product))
    z = np.sqrt(np.sum(lsumy_product))
    print(str(tree1))
    print(str(tree2))
    return( (x/(y*z))**2)
    
Pearson_evaluation(x[2][2],x[2][2])



add(mul(add(thickness, size), max(max(max(4, size), max(bc, thickness)), mul(max(bc, thickness), max(size, thickness)))), max(max(add(add(bc, size), max(shape, se)), mul(mul(se, bn), mul(size, add(nn, bn)))), max(add(add(nn, nn), mul(bc, bn)), add(add(size, 4), max(mitoses, nn)))))
add(mul(add(thickness, size), max(max(max(4, size), max(bc, thickness)), mul(max(bc, thickness), max(size, thickness)))), max(max(add(add(bc, size), max(shape, se)), mul(mul(se, bn), mul(size, add(nn, bn)))), max(add(add(nn, nn), mul(bc, bn)), add(add(size, 4), max(mitoses, nn)))))


1.0

In [15]:
# list42=[]
# tree_no=0
# fitnessM=[]
# for t1 in range(10):
#     for t2 in range(10):
#         tree_no=tree_no+1
#         print('tree : '+str(tree_no))
#         function = gp.compile(str(x[t1][t2]), pset)
#         for i in range(30):
#             if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
#                 list42.append(4)
#             else:
#                 list42.append(2)
#         print("Batch 1 : ")
#         print(list42)
#         print("\n")

#         scaled_x_list = []
#         fitnessS=[]
#         for j in range(12):
#             tn=0
#             tp=0
#             fn=0
#             fp=0
#             x_train, x_test, y_train, y_test = train_test_split(x_array, y_list, test_size=0.955, random_state=random.randint(1,100),shuffle = True)
#             listRand=[]
#             scaled_X = scaler.fit_transform(x_train)
#             scaled_x_list.append(scaled_X)
#             for i in range(30):
#                 if(function(scaled_X[i,0],scaled_X[i,1],scaled_X[i,2],scaled_X[i,3],scaled_X[i,4],scaled_X[i,5],scaled_X[i,6],scaled_X[i,7],scaled_X[i,8])<0):
#                     listRand.append(2)
#                 else:
#                     listRand.append(4)
#             print("Batch "+str(j+2)+" : ")

#             for n in range(30):
#                 if(listRand[n]==y_train[n]):
#                     if(y_train[n]==2):
#                         tn=tn+1
#                     else:
#                         tp=tp+1
#                 else:
#                     if(listRand[n]==2):
#                         fn=fn+1
#                     else:
#                         fp=fp+1



#             print(listRand)
#             print(y_train)
#             print("FP : "+str(fp)+" \n "+ "FN : "+str(fn)+" \n "+ "TP : "+str(tp)+" \n "+"TN : "+str(tn)+" \n ")
#             prec=tp/(tp+fp)
#             recall=tp/(tp+fn)
#             fitness=(2*prec*recall)/(prec+recall)
#             fitnessS.append(fitness)
#             print('Fitness = '+str(fitness*100))
#             print("\n")
#         fitnessM.append(fitnessS*100)
#         # print(x_train)
# print(scaled_x_list)

# print(fitnessM)


In [16]:
# scaler = preprocessing.StandardScaler()
# scaled_X_test = scaler.fit_transform(x_test)
# print(scaled_X_test)
# listRandtest=[]
# function = gp.compile(str(x[0][1]), pset)
# for i in range(len(y_test)-1):
#     if(function(scaled_X_test[i,0],scaled_X_test[i,1],scaled_X_test[i,2],scaled_X_test[i,3],scaled_X_test[i,4],scaled_X_test[i,5],scaled_X_test[i,6],scaled_X_test[i,7],scaled_X_test[i,8])<0):
#         listRandtest.append(2)
#     else:
#         listRandtest.append(4)
# print(listRandtest)
# print(y_test)

# tn1=0
# tp1=0
# fn1=0
# fp1=0


# for q in range(30):
#     if(listRandtest[q]==y_test[q]):
#         if(y_test[q]==2):
#             tn1=tn1+1
#         else:
#             tp1=tp1+1
#     else:
#         if(listRand[q]==2):
#             fn1=fn1+1
#         else:
#             fp1=fp1+1


           
# print("FP : "+str(fp1)+" \n "+ "FN : "+str(fn1)+" \n "+ "TP : "+str(tp1)+" \n "+"TN : "+str(tn1)+" \n ")
# prec1=tp1/(tp1+fp1)
# recall1=tp1/(tp1+fn1)
# fitness1=(2*prec1*recall1)/(prec1+recall1)
# print('Fitness = '+str(fitness1*100))


In [17]:
import deap
x = toolbox.population()
for t1 in range(10):
    for t2 in range(9):
        ind1 = x[t1][t2]; ind2 = x[t1][t2+1]
        c=[]
        c=deap.gp.cxOnePoint(ind1,ind2)
        print(str(c[1]))
# for i in range(10):
#     for j in range(9):
#         child1, child2 = [toolbox.clone(ind) for ind in (x[i][j], x[i][j+1])]
#         tools.cxBlend(child1, child2, 0.5)
#         del child1.fitness.values
#         del child2.fitness.values
#         print(child1);print(child2)

mul(mul(max(ma, shape), add(shape, 4)), add(max(mul(4, mitoses), ma), max(bn, ma)))
add(mul(max(size, size), mul(add(shape, 4), mitoses)), mul(max(4, ma), add(size, bn)))
max(add(add(shape, 4), mul(4, shape)), add(mul(nn, max(4, ma)), mul(nn, bn)))
mul(max(add(mul(max(4, bn), mul(bc, thickness)), max(max(size, nn), add(4, size))), max(add(mul(4, nn), add(size, ma)), add(mul(mitoses, 4), max(add(shape, 4), bn)))), mul(max(max(mul(ma, se), max(ma, ma)), mul(add(mitoses, thickness), add(mitoses, ma))), add(max(mul(ma, thickness), add(se, shape)), max(max(nn, nn), mul(4, 4)))))
add(mul(add(se, mitoses), max(bc, size)), add(mul(se, mitoses), add(max(shape, 4), mitoses)))
mul(max(max(add(max(bc, bc), mul(bn, ma)), mitoses), max(max(se, shape), max(mul(se, bc), bc))), max(add(add(add(4, size), se), mul(max(max(shape, 4), thickness), mitoses)), max(max(ma, nn), thickness)))
max(add(mul(ma, nn), thickness), add(mul(thickness, bc), add(thickness, se)))
max(se, max(add(ma, add(bn, size)), max(4, 

In [18]:

# Tree_fitness(x[1][1])

In [19]:
Tree_fitness(x[1][1])

TypeError: Tree_fitness() missing 2 required positional arguments: 'x' and 'y'

In [24]:
import time
time.time()

1543386464.8210907